In [8]:
# standard python utilities
import os
import sys
from os.path import basename, dirname, join, exists
import glob
import time

import pandas as pd
import numpy as np
import numpy.ma as ma
# from scipy.stats import gmean

# standard python plotting utilities
# import matplotlib as mpl
# import matplotlib.pyplot as plt

# standard geospatial python utilities
import geopandas as gpd
from osgeo import gdal
import rasterio

# mapping utilities
# import contextily as ctx
# from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
# import matplotlib.font_manager as fm

In [2]:
doc_dir = os.getcwd()
while basename(doc_dir) != 'Documents':
    doc_dir = dirname(doc_dir)
# dir of all gwfm data
gwfm_dir = dirname(doc_dir)+'/Box/research_cosumnes/GWFlowModel'
# dir of stream level data for seepage study
proj_dir = gwfm_dir + '/Oneto_Denier/'
dat_dir = proj_dir+'Stream_level_data/'

sfr_dir = gwfm_dir+'/SFR_data/'
uzf_dir = gwfm_dir+'/UZF_data/'

In [19]:
def add_path(fxn_dir):
    """ Insert fxn directory into first position on path so local functions supercede the global"""
    if fxn_dir not in sys.path:
        sys.path.insert(0, fxn_dir)

# other functions
py_dir = join(doc_dir,'GitHub/CosumnesRiverRecharge/python_utilities')
add_path(py_dir)

from mf_utility import get_layer_from_elev
from map_cln import gdf_bnds, plt_cln

# Load general files

In [3]:

dem_data = np.loadtxt(gwfm_dir+'\DIS_data\dem_52_9_200m_mean.tsv')



In [4]:
# Load model grid as geopandas object
grid_p = gpd.read_file(gwfm_dir+'/DIS_data/grid/grid.shp')


In [12]:
cell_area = grid_p.area.iloc[0]


In [5]:
grid_elev = gpd.read_file(join(gwfm_dir,'DIS_data','grid_elevation_m_statistics.shp'))

In [14]:
grid_sfr = gpd.read_file(sfr_dir+'/final_grid_sfr/grid_sfr.shp')


## Prepare Lake bathymetry

In [6]:
lak_shp = join(gwfm_dir,'LAK_data/floodplain_delineation')
# ifp = gpd.read_file(join(lak_shp,'inner_floodplain_domain/inner_floodplain_domain.shp' )).to_crs('epsg:32610')
# lfp = gpd.read_file(join(lak_shp,'lower_floodplain_approximate_area/lower_floodplain_approximate_area.shp' )).to_crs('epsg:32610')
lak_extent = gpd.read_file(join(lak_shp,'LCRFR_ModelDom_2017/LCRFR_2DArea_2015.shp' )).to_crs('epsg:32610')

fp_logger = pd.read_csv(join(gwfm_dir,'LAK_data','floodplain_logger_metadata.csv'))
fp_logger = gpd.GeoDataFrame(fp_logger, geometry = gpd.points_from_xy(fp_logger.Easting, fp_logger.Northing), crs='epsg:32610')
# find grid cell it is within
fp_grid = gpd.sjoin(fp_logger, grid_p, how='left',predicate='within')

In [9]:
fn = join(lak_shp,"floodplain_crop.tif")
if not exists(fn):
    # create clipped raster of just lake area
    dem_dir = join(gwfm_dir,'DEM_data')
    raster_name = dem_dir+'/mwt_peri_2_3.tif/mwt_peri_2_3_clipped.tif'
    import rasterio.mask
    with rasterio.open(raster_name) as src:
        out_image, out_transform = rasterio.mask.mask(src, lak_extent.geometry.values, crop=True)
        out_meta = src.meta
    # write output
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open(join(lak_shp,"floodplain_crop.tif"), "w", **out_meta) as dest:
        dest.write(out_image)

# prepare bathymetry file
lakeRst = rasterio.open(join(lak_shp,"floodplain_crop.tif"))
lakeBottom = lakeRst.read(1)
noDataValue = np.copy(lakeBottom[0,0])
#replace value for np.nan
lakeBottom[lakeBottom==noDataValue]= np.nan

# get raster minimum and maximum 
minElev = np.nanmin(lakeBottom)
maxElev = np.nanmax(lakeBottom)
print('Min bottom elevation %.2f m., max bottom elevation %.2f m.'%(minElev,maxElev))

# steps for calculation
nSteps = 151
# lake bottom elevation intervals
elevSteps = np.round(np.linspace(minElev,maxElev,nSteps),2)

# definition of volume function
def calculateVol_A(elevStep,elevDem,lakeRst, conv=1):
    tempDem = elevStep - elevDem[elevDem<elevStep]
    tempArea = len(tempDem)*lakeRst.res[0]*conv*lakeRst.res[1]*conv
    tempVol = tempDem.sum()*lakeRst.res[0]*conv*lakeRst.res[1]*conv
    return(tempVol, tempArea)
# calculate volumes, areas for each elevation
volArray = [0]
saArray = [0]
for elev in elevSteps[1:]:
    tempVol,tempArea = calculateVol_A(elev,lakeBottom,lakeRst)
    volArray.append(tempVol)
    saArray.append(tempArea)

# print("Lake bottom elevations %s"%elevSteps)
volArrayMCM = round(volArray[-1]/1000000,2) 
print("Lake volume in million of cubic meters %s"%volArrayMCM)

Min bottom elevation 3.03 m., max bottom elevation 11.57 m.
Lake volume in million of cubic meters 9.52


In [13]:
# identify grid cells in the lake area
lak_grid = gpd.overlay(grid_p, lak_extent[['OID_','geometry']], how='intersection')
# check if more than 50% of cell is covered by the lake, avoid conflicts with sfr
lak_grid = lak_grid.loc[lak_grid.geometry.area > (cell_area*0.5)]


In [27]:
# remove lake grid cells that overlap with sfr cells
lak_grid_cln = lak_grid.join(grid_sfr.set_index(['row','column'])[['reach']], on=['row','column'])
lak_grid_cln = lak_grid_cln[lak_grid_cln.reach.isna()]

lak_grid_cln.to_file(join(lak_shp, 'lak_grid_cln.shp'))
# lak_row, lak_col = lak_grid_cln.row.values-1, lak_grid_cln.column.values-1

# the lakarr and bdlknc are dependent on the model layer and geology 
# and so are not defined here


In [26]:
# lake stage (elevation), volume, and area (3 numbers per line)
bathtxt = np.column_stack((elevSteps, volArray, saArray))
np.savetxt(join(gwfm_dir,'LAK_data', 'oneto-denier.bath'), bathtxt, delimiter = '\t')
